In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from feat import Detector
from IPython.display import Image
from feat.utils import FEAT_EMOTION_COLUMNS

detector = Detector(device="cpu")

# Define the root directory of your dataset
dataset_dir = "DiffusionFER/DiffusionFER/DiffusionEmotion_S"
aus_csv_path = os.path.join(os.getcwd(), "DiffusionFER/DiffusionFER/DiffusionEmotion_S/result.csv")
results=[]
# Iterate through each emotion folder
folders = [f for f in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, f))]
#print(f"Emotion folders found: {folders}")

face_tracker = cv2.CascadeClassifier("frontal_face_features.xml") # loading a pre-trained face detection classifier from an XML file

# Loop through each emotion folder
for emotion in folders:
    emotion_folder = os.path.join(dataset_dir, emotion)
    image_files = [f for f in os.listdir(emotion_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # Process each image in the folder
    for image_file in image_files:
        image_path = os.path.join(emotion_folder, image_file)

        frame = cv2.imread(image_path)  # Load the image using OpenCV (as BGR)

        faces = detector.detect_faces(frame) # Detects face and returns coordinates
        landmarks = detector.detect_landmarks(frame, faces) # Detects facial landmarks
        emotions = detector.detect_emotions(frame, faces, landmarks) # analyses face landmarks and predicts emotions expressed
        aus = detector.detect_aus(frame, landmarks)

        # assume only one face captured
        faces = faces[0]
        landmarks = landmarks[0]
        emotions = emotions[0]  # Convert the first element of emotions to a numpy array
        aus=aus[0]

        #strongest_emotion = emotions.argmax(axis=1)

        for face, landmark, emotion, au_activation in zip(faces, landmarks, emotions,aus):

            (x0, y0, x1, y1, p) = face

            # cv2.rectangle(frame, (int(x0), int(y0)), (int(x1), int(y1)), (0, 0, 255), 3)
            # cv2.putText(frame, FEAT_EMOTION_COLUMNS[top_emo], (int(x0), int(y0 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

            # Remove the file extension
            filename = os.path.splitext(image_file)[0]
            result = {'file': image_file }
            result.update({f"AU_{j}": au for j, au in enumerate(au_activation)})
            results.append(result)

results_df = pd.DataFrame(results)
results_df.to_csv(aus_csv_path, index=False)
print(f"AU activations saved to {aus_csv_path}")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
dataset_sheet=os.path.join(os.getcwd(), "DiffusionFER/DiffusionFER/DiffusionEmotion_S/dataset_sheet.csv")
dataset_sheet = pd.read_csv(dataset_sheet)
# Extract the filenames from the 'subDirectory_filePath' column in dataset_sheet
dataset_sheet['file'] = dataset_sheet['subDirectory_filePath'].apply(lambda x: x.split('/')[-1])

# Merge the two DataFrames on the 'file' column
merged_data = pd.merge(results_df, dataset_sheet, on='file', how='inner')

# Save the merged DataFrame to a new CSV file
merged_data.to_csv(aus_csv_path, index=False)

In [ ]:
import matplotlib.pyplot as plt
# Classify the data based on valence values
merged_data['valence'] = pd.to_numeric(merged_data['valence'], errors='coerce')
positive_data = merged_data[merged_data['valence'] > 0]
negative_data = merged_data[merged_data['valence'] < 0]
neutral_data = merged_data[merged_data['valence'] == 0]  # Neutral data

# Extract Action Unit columns
au_columns = [col for col in merged_data.columns if col.startswith('AU')]

# Calculate mean for each condition
positive_means = positive_data[au_columns].mean()
negative_means = negative_data[au_columns].mean()
neutral_means = neutral_data[au_columns].mean()

# Print the means for debugging or analysis
print("Positive AUs Mean:")
print(positive_means)
print("\nNegative AUs Mean:")
print(negative_means)
print("\nNeutral AUs Mean:")
print(neutral_means)

# Calculate absolute differences (positive vs. negative)
abs_diff_pos_neg = (positive_means - negative_means).abs()
abs_diff_pos_neu = (positive_means - neutral_means).abs()
abs_diff_neg_neu = (negative_means - neutral_means).abs()

# Sort the differences for each comparison
sorted_diff_pos_neg = abs_diff_pos_neg.sort_values(ascending=False)
sorted_diff_pos_neu = abs_diff_pos_neu.sort_values(ascending=False)
sorted_diff_neg_neu = abs_diff_neg_neu.sort_values(ascending=False)

# Plot absolute differences
plt.figure(figsize=(12, 8))
plt.scatter(sorted_diff_pos_neg.index, sorted_diff_pos_neg.values, color='blue', label='Positive vs Negative')
plt.scatter(sorted_diff_pos_neu.index, sorted_diff_pos_neu.values, color='green', label='Positive vs Neutral')
plt.scatter(sorted_diff_neg_neu.index, sorted_diff_neg_neu.values, color='red', label='Negative vs Neutral')
plt.xticks(rotation=90)
plt.xlabel('Action Units (AUs)')
plt.ylabel('Absolute Difference of Means')
plt.title('Absolute Difference in AU Means Across Conditions')
plt.legend()

# Save the graph to the folder
output_path = os.path.join(os.getcwd(), "DiffusionFER/DiffusionFER/DiffusionEmotion_S/au_visualization.png")
plt.savefig(output_path, format='png')  # Save the graph
plt.show()
print(f"Graph saved to {output_path}")
